In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from xcube_geodb.core.geodb import GeoDBClient
import math
import os
import dotenv
import json
from tqdm import tqdm

dotenv.load_dotenv()

# load server information and credentials from environment
geodb = GeoDBClient()


In [ ]:
geodb.whoami

In [ ]:
geodb.get_my_collections()

In [ ]:
geodb.get_my_collections(database='eotdl')

In [ ]:
from eotdl.curation.stac import STACDataFrame

stac_file = 'example_data/EuroSAT-RGB-small-STAC/catalog.json'

gdf = STACDataFrame.from_stac_file(stac_file)
gdf

In [ ]:
gdf = gdf.convert_dtypes()

In [ ]:
# keep only the features which id starts with 'AnnualCrop'
filtered_gdf = gdf[gdf['id'].str.startswith('AnnualCrop')]
filtered_gdf.head()



In [ ]:
# keep only the labels
labels = gdf[gdf['collection'] == 'labels']
labels.head()


In [ ]:
def translate(o):
	if o == 'object' or str(o) == "string":
		return 'varchar'
	elif o == 'int64' or o == 'int32':
		return 'integer'
	elif o == 'float64' or o == 'float32':
		return 'float'
	return None

def translate_properties(gdf):
	properties = {}
	for k in gdf.dtypes.keys():
		if translate(gdf.dtypes[k]):
			properties[k] = translate(gdf.dtypes[k])
	return properties

In [ ]:
database = 'eotdl' # cannot use a different name for the database (try create database first)
collection_name = 'eurosat-test'

gdf = gdf.replace(math.nan, None)

# rename id for geodb
gdf.rename(columns={"id": "stac_id"}, inplace=True)


In [ ]:
if not geodb.database_exists(database):
    geodb.create_database(database)
if geodb.collection_exists(collection_name, database=database):
	geodb.drop_collection(collection_name, database=database)

geodb.create_collection(collection_name, translate_properties(gdf), database=database)

geodb.insert_into_collection(collection_name, values=gdf, database=database)

In [ ]:
geodb.get_collection(collection_name, database=database)

In [ ]:
geodb.get_collection(collection_name, database=database, query="type=eq.Feature")

In [ ]:
geodb.get_properties(collection_name, database=database)

Questions:

- Can we query if all the columns are of type json?
	- is technically possible, but not recommended (better to use the actual types or strings)
- Can we query with nested dicts?
	- we can store the nested dicts as strings and query them as such
- Can we do spatial queries using geometry?
	- `geodb.get_collection_by_bbox` (for arbitrary shapes, get the bbox first)
- How does the STAC API work?
	- every entry in a database, is a STAC item (feature), with its columns being the properties
	- catalogs / collections are built around them
	- We cannot use geodb STAC API, we need to enable an endppoint on eotdl API (stac compliant)

Actions:

- We should define the types of the columns in the metadata properly (for example, read some or all the data to infer the types)
- Explore querying the json format
- Add STAC API endpoint to eotdl API (instead of proxy the geodb STAC API)

Otherwise, we load the data into a pandas dataframe and query there.

In [ ]:
from eotdl.datasets import download_dataset

# download dataset

download_dataset('EuroSAT-Q1-small', path='data/output')


In [ ]:
from eotdl.curation.stac import STACDataFrame

# load metadata

stac_file = 'data/output/EuroSAT-Q1-small/v1/EuroSAT-Q1-small/catalog.json'

gdf = STACDataFrame.from_stac_file(stac_file)
gdf


In [ ]:
# filter metadata locally

filtered_gdf = gdf[gdf['id'].str.startswith('AnnualCrop')]
filtered_gdf

In [ ]:
from eotdl.datasets import download_file_url

# download only filtered assets

for asset in tqdm(filtered_gdf['assets'], desc='Downloading assets'):
	for k, v in asset.items():
		# print(k, v['href'])
		download_file_url(v['href'], 'data/output/EuroSAT-Q1-small/v1') # will store in path / assets

In [ ]:
!rm -rf data/output/EuroSAT-Q1-small/v1/assets

In [ ]:
labels = gdf[gdf['collection'] == 'labels']
labels.head()

In [ ]:
for asset in tqdm(labels['assets'], desc='Downloading labels'):
	for k, v in asset.items():
		# print(k, v['href'])
		download_file_url(v['href'], 'data/output/EuroSAT-Q1-small/v1') 

In [ ]:
# in order to filter by a particular label, we need to read the label item's assets...

label_assets = os.listdir('data/output/EuroSAT-Q1-small/v1/assets')
filtered_label_assets = []
for asset in tqdm(label_assets, desc='Downloading label assets'):
	try:
		data = json.load(open(f'data/output/EuroSAT-Q1-small/v1/assets/{asset}'))
		label = data['features'][0]['properties']['label']
		if label == 'AnnualCrop':
			filtered_label_assets.append(asset)
	except Exception as e:
		print('error in ', asset)
		continue

filtered_label_assets